**Computer vision mini project**(2020/E/031,2020/E/120)

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


01. Dataset prepartation

 We use raw data set of one video from https://www.kaggle.com/datasets/raajanwankhade/oep-dataset. From that dataset we get one video of wear camera video. Trim that video into small part. Phone and book useing portion. After that cange video in photo by extarcting frame from video file.

In [2]:
import cv2
import os

# Create directories for extracted frames
os.makedirs('/content/drive/MyDrive/computer vision/dataset3/output/phone_frames', exist_ok=True)
os.makedirs('/content/drive/MyDrive/computer vision/dataset3/output/book_frames', exist_ok=True)

# Function to extract frames every 3 seconds
def extract_frames(video_path, output_folder, frame_rate=1):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second of the video
    frame_interval = fps * 0.1  # Capture 1 frame every 1 seconds

    count = 0
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            cv2.imwrite(f"{output_folder}/frame_{frame_count}.jpg", frame)
            frame_count += 1
        count += 1
    cap.release()

# Example usage (replace with your paths)
extract_frames('/content/drive/MyDrive/computer vision/dataset3/phone video/2.avi', '/content/drive/MyDrive/computer vision/dataset3/output/phone_frames')
extract_frames('/content/drive/MyDrive/computer vision/dataset3/book video/1.avi', '/content/drive/MyDrive/computer vision/dataset3/output/book_frames')

02. Spit the Data set into train, validatin, test

In [3]:
import shutil
from sklearn.model_selection import train_test_split

def split_data(source, train_dest, val_dest, test_dest, test_size=0.1, val_size=0.1):
    files = os.listdir(source)
    train_files, test_files = train_test_split(files, test_size=test_size)
    train_files, val_files = train_test_split(train_files, test_size=val_size)

    for file in train_files:
        shutil.copy(os.path.join(source, file), train_dest)
    for file in val_files:
        shutil.copy(os.path.join(source, file), val_dest)
    for file in test_files:
        shutil.copy(os.path.join(source, file), test_dest)

# Create directories
for folder in ['train', 'val', 'test']:
    os.makedirs(f'/content/drive/MyDrive/computer vision/dataset3/output/dataset/phone/{folder}', exist_ok=True)
    os.makedirs(f'/content/drive/MyDrive/computer vision/dataset3/output/dataset/book/{folder}', exist_ok=True)

# Apply splitting
split_data('/content/drive/MyDrive/computer vision/dataset3/output/phone_frames', '/content/drive/MyDrive/computer vision/dataset3/output/dataset/phone/train', '/content/drive/MyDrive/computer vision/dataset3/output/dataset/phone/val', '/content/drive/MyDrive/computer vision/dataset3/output/dataset/phone/test')
split_data('/content/drive/MyDrive/computer vision/dataset3/output/book_frames', '/content/drive/MyDrive/computer vision/dataset3/output/dataset/book/train', '/content/drive/MyDrive/computer vision/dataset3/output/dataset/book/val', '/content/drive/MyDrive/computer vision/dataset3/output/dataset/book/test')


03. MobilNet model structure

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the MobileNet model for classification
def create_mobilenet_model():
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights=None)
    base_model.trainable = True

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_mobilenet_model()
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,210 (9.24 MB)

 Trainable params: 2,388,098 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

4. Train the model


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to your training and validation data
train_data_dir = '/content/drive/MyDrive/computer vision/dataset3/output/dataset'
val_data_dir = '/content/drive/MyDrive/computer vision/dataset3/output/dataset'

# Create ImageDataGenerator for data augmentation and loading
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)  # Only normalize for validation

# Load and prepare training data
train_data = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Resize images to match MobileNet input
    batch_size=32,
    class_mode='binary'  # Use 'categorical' if you have more than 2 classes
)

# Load and prepare validation data
val_data = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Use 'categorical' if you have more than 2 classes
)

# Train the model
model.fit(train_data, validation_data=val_data, epochs=10)
model.save('/content/drive/MyDrive/computer vision/dataset3/output/mobilenet_model.h5')

Found 418 images belonging to 2 classes.
Found 418 images belonging to 2 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 111s 3s/step - accuracy: 0.7018 - loss: 0.5637 - val_accuracy: 0.7775 - val_loss: 0.6833
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 64s 411ms/step - accuracy: 0.9715 - loss: 0.1163 - val_accuracy: 0.7775 - val_loss: 0.6534
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 625ms/step - accuracy: 0.9658 - loss: 0.0675 - val_accuracy: 0.7775 - val_loss: 0.6120
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 531ms/step - accuracy: 0.9875 - loss: 0.0577 - val_accuracy: 0.7775 - val_loss: 0.5738
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 495ms/step - accuracy: 0.9190 - loss: 0.2022 - val_accuracy: 0.7775 - val_loss: 0.5721
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 474ms/step - accuracy: 0.9722 - loss: 0.0827 - val_accuracy: 0.7775 - val_loss: 0.5447
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 635ms/step - accuracy: 0.9411 - loss: 0.1304 - val_accuracy: 0.7775 - val_loss: 0.5313
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 483ms/step - accuracy: 0.9838 - loss: 0.0406 - val_accuracy: 0.7775 

In [6]:
print(train_data.class_indices)


{'book': 0, 'phone': 1}


5. Evaluvation and testing
For testing purposes, we have already extracted a video. When testing that video, it starts extracting frames and saving them in a directory. The model checks the frame images, and if a frame includes a phone or book, that frame image is saved in a separate directory. Otherwise, no image file is saved.

In [7]:
import cv2
import tensorflow as tf
import os

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/computer vision/dataset3/output/mobilenet_model.h5')

# Correct label indices (Verify your class mapping)
class_names = ["Book", "Phone"]  # Ensure this matches your dataset order

# Create directories for saving results
phone_folder = '/content/drive/MyDrive/computer vision/classified_frames/phone'
book_folder = '/content/drive/MyDrive/computer vision/classified_frames/book'
os.makedirs(phone_folder, exist_ok=True)
os.makedirs(book_folder, exist_ok=True)

def extract_and_classify_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (224, 224))
        normalized_frame = resized_frame / 255.0  # Ensure same normalization as training

        # Predict the class probabilities
        prediction = model.predict(normalized_frame.reshape(1, 224, 224, 3))
        predicted_class = tf.argmax(prediction, axis=1).numpy()[0]

        # Assign labels based on predictions
        label = class_names[predicted_class]

        # Save the frame in the appropriate folder
        frame_filename = f"frame_{frame_count}.jpg"
        save_path = phone_folder if label == "Phone" else book_folder
        cv2.imwrite(os.path.join(save_path, frame_filename), frame)

        frame_count += 1

    cap.release()
    print(f"Classification complete! Processed {frame_count} frames.")

# Run the updated classification
extract_and_classify_frames('/content/drive/MyDrive/computer vision/dataset3/phone video/4.avi')


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━

In [8]:
# Check if any images were saved
phone_images = os.listdir(phone_folder)
book_images = os.listdir(book_folder)

if len(phone_images) == 0 and len(book_images) == 0:
    print('No cheating behavior detected.')
else:
    print('Cheating behavior detected.')

Cheating behavior detected.
